In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # pour faire l'analyse de donnée et dessiner le tableau.
import seaborn as sb # pour dessiner les graphes pour visualiser la data. 

import seaborn as sb
import re
import string
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.corpus import stopwords
from nltk.util import ngrams
stop=set(stopwords.words('english'))
from collections import Counter


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib
import matplotlib.patches as mpatches
%matplotlib inline
import seaborn as sns; sns.set(style='white')
from mpl_toolkits.mplot3d import Axes3D

from sklearn.decomposition import PCA, TruncatedSVD
import math
import pickle

from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import *
import tokenizers
from sklearn.model_selection import StratifiedKFold

pd.set_option('max_colwidth', 40)

In [ ]:
train_data = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/tweet-sentiment-extraction/test.csv")
test_data.head()

Analyse des tweet par sentiments entre neutral, positive et negative pour voir la représentation.  

In [ ]:

f, axes = plt.subplots(1, 2,figsize=(15,5))


sb.countplot(x='sentiment',data=train_data,order=train_data.sentiment.value_counts().index,ax=axes[0])\
.set_title('Nombre d_apparition des différents sentiments dans training data');

sb.countplot(x='sentiment',data=test_data,order=test_data.sentiment.value_counts().index,ax=axes[1]);
plt.title('Nombre d_apparition des différents sentiments dans test data');

Il y'a + de neutral ce qui peut créer un biais dans l'échantillon (nous pensons ?) même si la distribution est la même entre training et data. 

On utilise Jaccard pour comparer la sélection du tweet du training data et celui sélectionné dans le test dans l'objectif de se rapprocher de 1, ce qui voudrait dire que les 2 sélections seraient équivalentes (auraient les mêmes mots dans la même quantité).

Le code est donné sur la page de la compétition.

In [ ]:
def jaccard_score(x):
 str1, str2 = str(x[0]),str(x[1])
 a = set(str1.lower().split())
 b = set(str2.lower().split())
 c = a.intersection(b)
 return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
train_data['Num_words_ST'] = train_data['selected_text'].apply(lambda x:len(str(x).split())) #Number Of words in Selected Text
train_data['Num_word_text'] = train_data['text'].apply(lambda x:len(str(x).split())) #Number Of words in main text
train_data['difference_in_words'] = train_data['Num_word_text'] - train_data['Num_words_ST'] #Difference in Number of words text and Selected Text

In [ ]:
train_data.head()

In [ ]:
train_data['jaccard'] = train_data[['text','selected_text']].progress_apply(jaccard_score,axis=1)

Ici on met un code pour nettoyer les tweets à savoir enlever les liens, les ponctuations, les crochets, les mots avec nombres collés sur eux pour éviter les abréviations américains comme m8 pour mate, nous avons mis en commentaire le \n car nous ne comprenons pas son intérêt. 

In [ ]:
def clean_text(text):
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text) #enlève les corchets
    text = re.sub('https?://\S+|www\.\S+', '', text) #enlève les liens 
    text = re.sub('<.*?>+', '', text) #enlève le texte à l'intérieur des crochets
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)#les nombres collés aux mots 
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text) #enlève la ponctuation
    return text


In [ ]:
train_data['text'] = train_data['text'].apply(lambda x:clean_text(x))
train_data['selected_text'] = train_data['selected_text'].apply(lambda x:clean_text(x))

In [ ]:
train_data.head()

Retirer les tweet où il n'y a pas de données. 

On voit que nos tweet sont débarassés des problèmes de lisibilités.

Maintenant, nous allons compter les mots les plus utilisés dans les tweets


In [ ]:
train_data['temp_list'] = train_data['selected_text'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_data['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Mots_communs','Nombre_d_apparition']
temp.style.background_gradient(cmap='Greens')

In [ ]:
fig = px.bar(temp, x="Nombre_d_apparition", y="Mots_communs", title='Les mots communs in Selected Text', orientation='h', 
             width=700, height=700,color='Mots_communs')
fig.show()

On va essayer de différencier les mots selon les sentiments. Ainsi, on va établir une relation dans un tableau pour le sentiement neutral,negatif et positif. Cependant, on doit avant supprimer les mots comme "a","the" qui sont parasites dans notre etude. 

In [ ]:
def remove_stopword(x):
    return [y for y in x if y not in stopwords.words('english')]
train_data['temp_list'] = train_data['temp_list'].apply(lambda x:remove_stopword(x))

In [ ]:
top = Counter([item for sublist in train_data['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Mots_communs','Nombre_d_apparition']
temp.style.background_gradient(cmap='Blues')

Examinons également les mots les plus courants dans text contenu dans le fichier test.csv

In [ ]:
train_data['temp_list1'] = train_data['text'].apply(lambda x:str(x).split()) #Liste de mots dans chaque ligne pour le texte
train_data['temp_list1'] = train_data['temp_list1'].apply(lambda x:remove_stopword(x)) #Supprimer les mots vides

In [ ]:
top = Counter([item for sublist in train_data['temp_list1'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Mots_communs','Nombre_d_apparition']
temp.style.background_gradient(cmap='Blues')

Mots les plus courants par sentiments :

In [ ]:
Positive_sent = train_data[train_data['sentiment']=='positive']
Negative_sent = train_data[train_data['sentiment']=='negative']
Neutral_sent = train_data[train_data['sentiment']=='neutral']

In [ ]:
#Les mots pour le sentiment positif 
top = Counter([item for sublist in Positive_sent['temp_list'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Mots_communs','Nombre_d_apparition']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
#Les mots pour le sentiment négatif 
top = Counter([item for sublist in Negative_sent['temp_list'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Mots_communs','Nombre_d_apparition']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
#Les mots pour le sentiment neutre
top = Counter([item for sublist in Neutral_sent['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Mots_communs','Nombre_d_apparition']
temp_neutral.style.background_gradient(cmap='Greys')

Au final, on remarque que certains mots apparaisent dans plusieurs sentiments. On cherchera à trouver les mots uniques de chaque sentiment.

In [ ]:
raw_text = [word for word_list in train_data['temp_list1'] for word in word_list]

In [ ]:
def words_unique(sentiment,numwords,raw_words):
    allother = []
    for item in train_data[train_data.sentiment != sentiment]['temp_list1']:
        for word in item:
            allother .append(word)
    allother  = list(set(allother ))
    
    specificnonly = [x for x in raw_text if x not in allother]
    
    mycounter = Counter()
    
    for item in train_data[train_data.sentiment == sentiment]['temp_list1']:
        for word in item:
            mycounter[word] += 1
    keep = list(specificnonly)
    
    for word in list(mycounter):
        if word not in keep:
            del mycounter[word]
    
    Unique_words = pd.DataFrame(mycounter.most_common(numwords), columns = ['mots','nombre_d_apparition'])
    
    return Unique_words

In [ ]:
#Unique mot pour le sentiment postif 
Unique_Positive= words_unique('positive', 20, raw_text)
print("The top 20 unique words in Positive Tweets are:")
Unique_Positive.style.background_gradient(cmap='Greens')

In [ ]:
#Unique mot pour le sentiment negatif
Unique_Negative= words_unique('negative', 10, raw_text)
print("The top 10 unique words in Negative Tweets are:")
Unique_Negative.style.background_gradient(cmap='Reds')

In [ ]:
#Unique mot pour le sentiment neutre
Unique_Neutral= words_unique('neutral', 10, raw_text)
print("The top 10 unique words in Neutral Tweets are:")
Unique_Neutral.style.background_gradient(cmap='Greys')

In [ ]:
train_data.isnull().sum(),test_data.isnull().sum()

À partir de maintenant on va "compter" le nombre de mots dans le texte du tweet et le texte sélectionné pour pouvoir appliquer l'indice de jaccard.

In [ ]:
train_data['text_length'] = train_data['text'].progress_apply(lambda x: len(str(x).split()))
train_data['selected_text_length'] = train_data['selected_text'].progress_apply(lambda x: len(str(x).split()))

In [ ]:
test_data['text_length'] = test_data['text']

On voit que nos tweet sont débarassés des problèmes de lisibilités. 

In [ ]:
g=sb.FacetGrid(data=train_data,col='sentiment',height=5);
g.map(sb.kdeplot,'selected_text_length');
plt.subplots_adjust(top=0.87,)
g.fig.suptitle('Distribution of selected_text length(no.of words) in training data for various sentiment labels',fontsize=16);

Ici il est intéressant de remarquer que les avis tranchés (positif ou négatif donc) nécessite beaucoup moins de mots pour s'en apercevoir, ce qui est intuitif, à l'inverse un sentiment neutre est plus souvent exprimé en plus de mots (atténuation, lexique mesuré, phrases en 2 parties avec pour et contre etc...). 

Ainsi on peut supposer que l indice de jacquart serait proche de 1 pour un tweet neutre car le nombre de mots sélectionnés est plus grand et donc englobe quasi l ensemble du tweet, là ou pour les tweets positifs/tranchés peu de mots suffisent donc le nombre de mots sélectionnés est plus faible.  